## Quiz 9 Answers

In [1]:
# Working with data:
import numpy  as np                                   # For scientific computing.
import pandas as pd                                   # Working with tables.

# Specific data providers:
from tiingo import TiingoClient                       # Stock prices.
import quandl                                         # Economic data, futures prices, ...

# API keys:
tiingo = TiingoClient({'api_key':'XXXX'})
quandl.ApiConfig.api_key = 'YYYY'

# Plotting:
import matplotlib.pyplot as plt                        # Basic plot library.
plt.style.use('ggplot')                                # Make plots look nice

Q4: 

In [2]:
PRICE = tiingo.get_dataframe(['SPY','GLD','TLT'], '2005-1-1', metric_name='adjClose')
PRICE.index = pd.to_datetime(PRICE.index).tz_convert(None)
RET = PRICE.pct_change()

In [3]:
ret = RET.loc['2009',['SPY','TLT','GLD']]

cov     = ret.cov() * 252
cov_inv = pd.DataFrame(np.linalg.inv(cov), columns=cov.columns, index=cov.index)

w_minVol = cov_inv.sum() / cov_inv.sum().sum()
w_minVol

SPY    0.281381
TLT    0.476143
GLD    0.242477
dtype: float64

Q5:

In [4]:
RET.loc['2009'].corr()

,SPY,GLD,TLT
SPY,1.000000,0.036112,-0.323306
GLD,0.036112,1.000000,0.099647
TLT,-0.323306,0.099647,1.000000


Q6:

In [5]:
RATES = quandl.get(['FRED/FEDFUNDS','FRED/DGS1']) / 100
RATES.columns = ['FedFunds','Treasury_1']

In [6]:
ret = RET.loc['2005':'2015',['SPY','TLT','GLD']]

cov     = ret.cov() * 252
cov_inv = pd.DataFrame(np.linalg.inv(cov), columns=cov.columns, index=cov.index)

r_annual       = ret.add(1).resample('A').prod().sub(1)
r_annual_Tbill = RATES.Treasury_1.resample('A').first()

rx_annual = r_annual.sub(r_annual_Tbill, 'rows').dropna()

meanx = rx_annual[['SPY','TLT','GLD']].mean()  # Risk premiums (average annual excess returns)

w_maxSharpe = cov_inv.dot(meanx) /  cov_inv.dot(meanx).sum()
w_maxSharpe

SPY    0.330818
TLT    0.505456
GLD    0.163726
dtype: float64

In [7]:
# Never rebalance: first compound, then weight
RET['2015':'2021-3-31'].add(1).cumprod().multiply(w_maxSharpe).sum('columns') * 1000

2015-01-02    1006.163453
2015-01-05    1010.699782
2015-01-06    1018.897370
2015-01-07    1020.866736
2015-01-08    1018.953983
                 ...     
2021-03-25    1575.116631
2021-03-26    1584.967154
2021-03-29    1576.458453
2021-03-30    1573.884282
2021-03-31    1576.721747
Length: 1572, dtype: float64

Q8:

In [9]:
RET = RET.dropna(subset=['SPY'])
RET

,SPY,GLD,TLT
2005-01-04,-0.012219,-0.006509,-0.010480
2005-01-05,-0.006901,-0.001638,0.005352
2005-01-06,0.005084,-0.012187,0.000680
2005-01-07,-0.001433,-0.007355,0.002264
2005-01-10,0.004728,0.002629,0.001581
...,...,...,...
2021-04-06,-0.000591,0.008029,0.006793
2021-04-07,0.001157,-0.002818,-0.006965
2021-04-08,0.004747,0.010752,0.008255
2021-04-09,0.007270,-0.007538,-0.003623


In [10]:
def get_rebalance_dates(frequency):
    group = getattr(PRICE.index, frequency) 
    return PRICE[:1].index.union(PRICE.groupby([PRICE.index.year, group]).tail(1).index)



def run_backtest(frequency):   
    
    rebalance_dates = get_rebalance_dates(frequency) 

    portfolio_value = pd.Series(1,                      index=[rebalance_dates[0]])    
    weights         = pd.DataFrame(columns=RET.columns, index=[rebalance_dates[0]])
    trades          = pd.DataFrame(columns=RET.columns, index=[rebalance_dates[0]])

    previous_positions = weights.iloc[0]
        
    for i in range(len(rebalance_dates)-1):
        start_date = rebalance_dates[i]
        end_date   = rebalance_dates[i+1]

        cum_ret = RET[start_date:end_date][1:].add(1).cumprod()
        
        start_weights = select_weights(start_date)      # Call "select_weights()" function to get the weights
            
        new_positions = portfolio_value.iloc[-1] * start_weights 
        
        start_to_end_positions = new_positions  * cum_ret
        start_to_end_value     = start_to_end_positions.sum('columns')

        portfolio_value = portfolio_value.append(start_to_end_value) 
                
        weights = weights.append(start_to_end_positions.div(start_to_end_value,'rows'))                 
                
        trades.loc[start_date] = new_positions - previous_positions 
        previous_positions     = start_to_end_positions.iloc[-1]      # Previous positions for the next rebalance round

    return portfolio_value, weights, trades




def select_weights(date):
    if not RET[:date].empty:                                          # If previous data available.
        cov     = RET[['SPY','TLT','GLD']][:date][-200:].cov() * 252  # Use most recent 200 returns up to currrent date.
        cov_inv = pd.DataFrame(np.linalg.inv(cov), columns=cov.columns, index=cov.index)

        w = cov_inv.sum() /  cov_inv.sum().sum()         # Minimum-volatility portfolio.
    else:
        w = pd.Series(1/3, index=['SPY','TLT','GLD'])    # If no previous data available: choose equal weights.
    return w                                             # Return the selected weights to the loop
    
    

min_vol, weights, trades = run_backtest('quarter')
min_vol[:'2020'] * 10000

2005-01-03    10000.000000
2005-01-05     9989.380138
2005-01-06     9967.933225
2005-01-07     9946.579491
2005-01-10     9976.181659
                  ...     
2020-12-24    41727.896113
2020-12-28    41800.731922
2020-12-29    41791.455892
2020-12-30    41930.809628
2020-12-31    42059.459358
Length: 4027, dtype: float64

Q9:

In [11]:
def select_weights(date):
    if not RET[:date].empty:                                          # If previous data available.
        
        meanx = pd.Series({'SPY':0.08, 'TLT':0.03, 'GLD':0.04})
        
        cov     = RET[['SPY','TLT','GLD']][:date][-200:].cov() * 252  # Use most recent 200 returns up to currrent date.
        cov_inv = pd.DataFrame(np.linalg.inv(cov), columns=cov.columns, index=cov.index)

        w = cov_inv.dot(meanx) /  cov_inv.dot(meanx).sum()
    else:
        w = pd.Series(1/3, index=['SPY','TLT','GLD'])    # If no previous data available: choose equal weights.
    return w                                             # Return the selected weights to the loop
    
    

min_vol, weights, trades = run_backtest('quarter')
min_vol[:'2020'] * 10000

2005-01-03    10000.000000
2005-01-05     9989.380138
2005-01-06     9967.933225
2005-01-07     9946.579491
2005-01-10     9976.181659
                  ...     
2020-12-24    40542.596940
2020-12-28    40660.744301
2020-12-29    40633.282163
2020-12-30    40748.021974
2020-12-31    40881.135485
Length: 4027, dtype: float64

Q 10:

In [12]:
def select_weights(date):
    if not RET[:date].empty:                                          # If previous data available.
        cov     = RET[['SPY','TLT','GLD']][:date][-50:].cov() * 252  # Use most recent 50 returns up to currrent date.
        cov_inv = pd.DataFrame(np.linalg.inv(cov), columns=cov.columns, index=cov.index)

        w = cov_inv.sum() /  cov_inv.sum().sum()         # Minimum-volatility portfolio.
    else:
        w = pd.Series(1/3, index=['SPY','TLT','GLD'])    # If no previous data available: choose equal weights.
    return w                                             # Return the selected weights to the loop
    
    

min_vol, weights, trades = run_backtest('month')
min_vol[:'2020'] * 10000

2005-01-03    10000.000000
2005-01-05     9989.380138
2005-01-06     9967.933225
2005-01-07     9946.579491
2005-01-10     9976.181659
                  ...     
2020-12-24    42514.309122
2020-12-28    42614.328141
2020-12-29    42578.120873
2020-12-30    42690.110138
2020-12-31    42810.593801
Length: 4027, dtype: float64

In [13]:
weights.loc['2018-5-21']

SPY    0.208243
GLD    0.287700
TLT    0.504057
Name: 2018-05-21 00:00:00, dtype: float64

In [14]:
min_vol.loc['2018-5-21'] * 10000  # portfolio value

32469.876154019275

In [15]:
# Multiply portfolio value by weights to get $ positions:
min_vol.loc['2018-5-21'] * weights.loc['2018-5-21'] * 10000

SPY     6761.635651
GLD     9341.579469
TLT    16366.661034
Name: 2018-05-21 00:00:00, dtype: float64